In [1]:
import pandas as pd

# 准备矩阵数据
users = ["user1", "user2", "user3", "user4", "user5"]
items = ["itemA", "itemB", "itemC", "itemD", "itemE"]
# 显性数据 0未购买，1已经购买
# 稠密矩阵
# 目的是补全【user1】对【itemE】的评分
data_set = [
    [5, 3, 4, 4, None],
    [3, 1, 2, 3, 3],
    [4, 3, 4, 3, 5],
    [3, 3, 1, 5, 4],
    [1, 5, 5, 2, 1]
]
df = pd.DataFrame(data_set, columns=items, index=users)
df

,itemA,itemB,itemC,itemD,itemE
user1,5,3,4,4,NaN
user2,3,1,2,3,3.0
user3,4,3,4,3,5.0
user4,3,3,1,5,4.0
user5,1,5,5,2,1.0


# 皮尔逊相似度
>对余弦相似度的优化，增加了长度相关
$$
r(A,B) = \frac {\sum\limits_{i=1}^n(A_i-\overline{A})\times (B_i-\overline{B})}{\sqrt{\sum\limits_{i=1}^n(A_i-\overline{A})^2} \times \sqrt{\sum\limits_{i=1}^n(B_i-\overline{B})^2 }}
$$


In [2]:
# 获取用户的皮尔逊相似度
user_similar = df.T.corr()
user_similar

,user1,user2,user3,user4,user5
user1,1.000000,0.852803,0.707107,0.000000,-0.792118
user2,0.852803,1.000000,0.467707,0.489956,-0.900149
user3,0.707107,0.467707,1.000000,-0.161165,-0.466569
user4,0.000000,0.489956,-0.161165,1.000000,-0.641503
user5,-0.792118,-0.900149,-0.466569,-0.641503,1.000000


## 基于用户的相似度预测
> 就是*相似用户*对*同一内容*评分的*加权平均数*
- U 用户集合 = users
- u 用户 = user1
- i 内容 = itemE
- $r_{ui}$ 用户u对内容i的评分
- $sim (u,v)$ 用户u，v相似 =user2,user3
- $v \in U$ U中的用户v
$$
pred(u,i) =\hat{r}_{ui}= \frac {\sum\limits_{v \in U} sim(u,v) \times r_{vi}} {\sum\limits_{v \in U}|sim(u,v)|}
$$
$$
r_{1E} = \frac {0.85 \times 3.0 + 0.71 \times 5.0} {0.85+0.71} = 3.91
$$

In [3]:
# 内容的相似度矩阵
item_similar = df.corr()
item_similar

,itemA,itemB,itemC,itemD,itemE
itemA,1.000000,-0.476731,-0.123091,0.532181,0.969458
itemB,-0.476731,1.000000,0.645497,-0.310087,-0.478091
itemC,-0.123091,0.645497,1.000000,-0.720577,-0.427618
itemD,0.532181,-0.310087,-0.720577,1.000000,0.581675
itemE,0.969458,-0.478091,-0.427618,0.581675,1.000000


# 基于内容的相似度预测
> 和基于用户的类似
> *相似物品*从*同一用户*评分的*加权平均数*
$$
r_{1E} = \frac {0.97 \times 5.0 + 0.58 \times 4.0} {0.85+0.97} = 4.63
$$

In [4]:
# 稀疏矩阵
data_set_sparse = [
    [5, 3, 4, None, None],
    [None, None, None, 3, 3],
    [4, 3, 4, None, 5],
    [None, 3, None, 5, None],
    [1, 5, 5, 2, 1]
]
df_sparse = pd.DataFrame(data_set_sparse, columns=items, index=users)
df_sparse

,itemA,itemB,itemC,itemD,itemE
user1,5.0,3.0,4.0,NaN,NaN
user2,NaN,NaN,NaN,3.0,3.0
user3,4.0,3.0,4.0,NaN,5.0
user4,NaN,3.0,NaN,5.0,NaN
user5,1.0,5.0,5.0,2.0,1.0


# 基于模型的预测
> 适用于稀疏矩阵
## 图模型
1. 构建二分图
![1610347747599](../../.assets/graph1.png)
2.根据两个顶点之间的*路径数*、*路径长度*、*经过的定点数* 来评价两个顶点的相关性
## 矩阵分解
> ALS 算法 [详情](../../ML.md)